# Covariance Matrix(3)
- - -

#### 1. Covariance Matrix - 학습했던 내용!!!

* Calculating the mean of Sample Vectors - HW7 

* Outer product of sample vectors - HW8 

* Covariance Matrix - HW8 

#### 2. 새로운 sample mean vector와 outer product를 구하는 방법으로 Covariance Matrix(HW8의 방법 3 적용) 구하기
* Calculating the mean of a vector & outer product with nan's
* ★Computing the mean together with the covariance★
- - -

#### pyspark import & SparkContext 생성

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType

sc = SparkContext(master="local[*]")
print(sc)

# Just like using Spark requires having a SparkContext, using SQL requires an SQLContext
sqlContext = SQLContext(sc)
print(sqlContext)

<SparkContext master=local[*] appName=pyspark-shell>


### 1. Calculating the mean of a vector & outer product with nan's
* We often get vectors $\mathbf{x}$ in which some, but not all, of the entries are `nan`. 
* We want to compute the mean of the elements of $\mathbf{x}$. 
* If we use `np.mean` we will get the result `nan`. 
* A useful alternative is to use `np.nanmean` which removes the `nan` elements and takes the mean of the rest.

#### (1) nan이란 무엇인가
* Not a Number의 약어
* **표현 불가능한 수치형 결과**를 나타내는 값

In [2]:
import numpy as np
a=np.array([1,np.nan,2,np.nan,3,4,5])
print('a=',a)
print('np.mean(a)=',np.mean(a)) # nan이 포함되어 있기 때문에 계산이 되지 않는다.

# nan을 특정한 수치로 처리하여 계산할 것인가? 아니면 빼고 계산할 것인가?
# 사용하고자 하는 방법에 따라 계산된 결과가 다르다!!!!!!!!!
print('np.mean(np.nan_to_num(a))=',np.mean(np.nan_to_num(a))) # =(1+0+2+0+3+4+5)/7
print('np.nanmean(a)=',np.nanmean(a)) # =(1+2+3+4+5)/5

a= [ 1. nan  2. nan  3.  4.  5.]
np.mean(a)= nan
np.mean(np.nan_to_num(a))= 2.142857142857143
np.nanmean(a)= 3.0


#### (2) nan과 관련된 함수들...
* np.nan* 으로 확인
* `np.nan`, `np.isnan`, `np nan_to_num`, `np.nansum`, `np.nanmean` 등등
* 목적에 따라 선택하여 사용할 것.

In [3]:
print("# np.nan을 이용하여 nan을 생성할 수 있다.")
print(np.nan)
print('nan list = ',[np.nan, np.nan, np.nan])

# np.nan을 이용하여 nan을 생성할 수 있다.
nan
nan list =  [nan, nan, nan]


In [4]:
print("# np.isnan을 이용하여 boolean 값을 획득할 수 있다(nan일 경우 True return).")
a = np.array([1,np.nan,2,np.nan,3,4,5])
print("a = ", a)
print("np.isnan(a) = ", np.isnan(a))

# np.isnan을 이용하여 boolean 값을 획득할 수 있다(nan일 경우 True return).
a =  [ 1. nan  2. nan  3.  4.  5.]
np.isnan(a) =  [False  True False  True False False False]


In [5]:
print("# nan_to_num을 이용하여 nan을 특정한 값(0)으로 바꿀 수 있다.")
print("a = ", a)
print("np.nan_to_num(a) = ", np.nan_to_num(a))

# nan_to_num을 이용하여 nan을 특정한 값(0)으로 바꿀 수 있다.
a =  [ 1. nan  2. nan  3.  4.  5.]
np.nan_to_num(a) =  [1. 0. 2. 0. 3. 4. 5.]


In [6]:
print("nan을 특정한 겂으로 처리한 후의 sum = ",sum(np.nan_to_num(a)))
print("nan을 바꾸지 않은 후의 sum = ", sum(a))

nan을 특정한 겂으로 처리한 후의 sum =  15.0
nan을 바꾸지 않은 후의 sum =  nan


**★★ isnan으로부터 생성된 nan의 True/False를 수치형으로 변환**

* `np.isnan(a) = array([False,  True, False,  True, False, False, False])`

* `1-np.isnan(a) = array([1, 0, 1, 0, 1, 1, 1])`  

In [7]:
print("np.isnan(a) = ",np.isnan(a))
print("1-np.isnan(a) = ",1 - np.isnan(a))
print("nan의 개수 파악하기 = ",sum(1-np.isnan(a)))

np.isnan(a) =  [False  True False  True False False False]
1-np.isnan(a) =  [1 0 1 0 1 1 1]
nan의 개수 파악하기 =  5


#### (3) Calculating the mean of a vector with nan's

In [8]:
sum(np.nan_to_num(a))/sum(1-np.isnan(a))

3.0

#### (4) Calculating the outer product of a vector with nan's

In [9]:
S =np.outer(a,a)
N =1-np.isnan(S)

print("outer product with nan\n->", S)
print("\n",N)

outer product with nan
-> [[ 1. nan  2. nan  3.  4.  5.]
 [nan nan nan nan nan nan nan]
 [ 2. nan  4. nan  6.  8. 10.]
 [nan nan nan nan nan nan nan]
 [ 3. nan  6. nan  9. 12. 15.]
 [ 4. nan  8. nan 12. 16. 20.]
 [ 5. nan 10. nan 15. 20. 25.]]

 [[1 0 1 0 1 1 1]
 [0 0 0 0 0 0 0]
 [1 0 1 0 1 1 1]
 [0 0 0 0 0 0 0]
 [1 0 1 0 1 1 1]
 [1 0 1 0 1 1 1]
 [1 0 1 0 1 1 1]]


### Computing the covariance  when there are `nan`s
The covariance is a mean of outer products.

We calculate two matrices:
* $S$ - the sum of the matrices, where `nan`->0
* $N$ - the number of not-`nan` element for each matrix location.

We then calculate the mean as $S/N$ (division is done element-wise)

- - - - - - - - - - - - - -

### 2. Computing the mean together with the covariance

To compute the covariance matrix we need to compute both $\hat{E}(\mathbf{x}^T \mathbf{x})$ and $\hat{E}(\mathbf{x})$. Using a simple trick, we can compute both at the same time.

- - -
(1) Step 1

Here is the trick: lets denote a $d$ dimensional **column vector** by $\mathbf{x} = (x_1,x_2,\ldots,x_d)$ (note that the subscript here is the index of the coordinate, not the index of the example in the training set as used above). 

The augmented vector $\mathbf{x}'$ is defined to be the $d+1$ dimensional vector $\mathbf{x}' = (1,x_1,x_2,\ldots,x_d)$.

- - -
(2) Step 2

The outer product of $\mathbf{x}'$ with itself is equal to 

$$ {\mathbf{x}'}^T {\mathbf{x}'}
= \left[\begin{array}{c|ccc}
    1 &  &{\mathbf{x}'} &\\
    \hline \\
    {\mathbf{x}'}^T & &{\mathbf{x}'}^T {\mathbf{x}'} \\ \\
    \end{array}
    \right]
$$

Where the lower left matrix is the original outer product ${\mathbf{x}'}^T{\mathbf{x}'}$ and the first row and the first column are ${\mathbf{x}'}$ and ${\mathbf{x}'}^T$ respectively.

- - -
(3) Step 3

Now suppose that we do the take the average of the outer product of the augmented vector and convince yourself that:
$$
\hat{E}({\mathbf{x}'}^T{\mathbf{x}'}) = \frac{1}{n} \sum_{i=1}^n {\mathbf{x}'}_i^T{\mathbf{x}'}_i
= \left[\begin{array}{c|ccc}
    1 &  &\hat{E}({\mathbf{x}'}) &\\
    \hline \\
    \hat{E}({\mathbf{x}'}^T) & &\hat{E}({\mathbf{x}'}^T{\mathbf{x}'}) \\ \\
    \end{array}
    \right]
$$

So indeed, we have produced the outer product average together with (two copies of) the average $\hat{E}({\mathbf{x}'}^T)$

**★★★★★여기서 n은 non-nan의 sum★★★★★**

- - -
(4) Step 4

Computing covariance matrix

\begin{align*}
\text{Cov}(\mathbf{x}) &= \hat E({\mathbf{x}'}^T{\mathbf{x}'}) \ - \hat{E}({\mathbf{x}'})^T\hat{E}({\mathbf{x}'})
\end{align*}

In [10]:
# in python
import numpy as np

print("★Step 1★\n")

sample1 = np.array([2,np.nan,3,5], dtype = 'float16')
sample2 = np.array([4,5,np.nan,10], dtype = 'float16')
sample3 = np.array([7,30,9,np.nan], dtype = 'float16')
print("각 sample의 값\n-> ", sample1, sample2, sample3)

# nan이 있는 sample에서 sample mean을 구하기 위한 trick 적용
sample1_trick = np.insert(sample1, 0, 1)
sample2_trick = np.insert(sample2, 0, 1)
sample3_trick = np.insert(sample3, 0, 1)
print("\ntrick이 적용된 각 sample의 값\n-> ", sample1_trick, sample2_trick, sample3_trick)

★Step 1★

각 sample의 값
->  [ 2. nan  3.  5.] [ 4.  5. nan 10.] [ 7. 30.  9. nan]

trick이 적용된 각 sample의 값
->  [ 1.  2. nan  3.  5.] [ 1.  4.  5. nan 10.] [ 1.  7. 30.  9. nan]


In [11]:
# outer product 구하기 
print("★Step 2★\n")

sample1_outer = np.outer(sample1_trick, sample1_trick)
sample2_outer = np.outer(sample2_trick, sample2_trick)
sample3_outer = np.outer(sample3_trick, sample3_trick)

print("sample1_trick의 outer product\n->", sample1_outer)
print("\nsample2_trick의 outer product\n->", sample2_outer)
print("\nsample3_trick의 outer product\n->", sample3_outer)

# outer product의 non-nan 개수 구하기
sample1_outer_nonnan = 1 - np.isnan(sample1_outer)
sample2_outer_nonnan = 1 - np.isnan(sample2_outer)
sample3_outer_nonnan = 1 - np.isnan(sample3_outer)

print("\nsample1_trick outer product의 non-nan\n->", sample1_outer_nonnan)
print("\nsample2_trick outer product의 non-nan\n->", sample2_outer_nonnan)
print("\nsample3_trick outer product의 non-nan\n->", sample3_outer_nonnan)

★Step 2★

sample1_trick의 outer product
-> [[ 1.  2. nan  3.  5.]
 [ 2.  4. nan  6. 10.]
 [nan nan nan nan nan]
 [ 3.  6. nan  9. 15.]
 [ 5. 10. nan 15. 25.]]

sample2_trick의 outer product
-> [[  1.   4.   5.  nan  10.]
 [  4.  16.  20.  nan  40.]
 [  5.  20.  25.  nan  50.]
 [ nan  nan  nan  nan  nan]
 [ 10.  40.  50.  nan 100.]]

sample3_trick의 outer product
-> [[  1.   7.  30.   9.  nan]
 [  7.  49. 210.  63.  nan]
 [ 30. 210. 900. 270.  nan]
 [  9.  63. 270.  81.  nan]
 [ nan  nan  nan  nan  nan]]

sample1_trick outer product의 non-nan
-> [[1 1 0 1 1]
 [1 1 0 1 1]
 [0 0 0 0 0]
 [1 1 0 1 1]
 [1 1 0 1 1]]

sample2_trick outer product의 non-nan
-> [[1 1 1 0 1]
 [1 1 1 0 1]
 [1 1 1 0 1]
 [0 0 0 0 0]
 [1 1 1 0 1]]

sample3_trick outer product의 non-nan
-> [[1 1 1 1 0]
 [1 1 1 1 0]
 [1 1 1 1 0]
 [1 1 1 1 0]
 [0 0 0 0 0]]


In [12]:
# outerproduct의 sum
print("★Step 3★\n")

sample_outer_sum = np.nansum(np.dstack([sample1_outer, sample2_outer]), axis=2)
sample_outer_sum = np.nansum(np.dstack([sample_outer_sum, sample3_outer]), axis=2)
print("sample outer product sum\n->", sample_outer_sum)

# non-name의 sum
sample_nonnan_sum = sample1_outer_nonnan + sample2_outer_nonnan + sample3_outer_nonnan
print("\nsample non-nan sum\n->", sample_nonnan_sum)

print("\nsample mean vector\n->", 
      sample_outer_sum[0, 1:] / np.float16(sample_nonnan_sum[0, 1:]))
print("\nouter product의 sum\n->",\
      sample_outer_sum[1:, 1:] / np.float16(sample_nonnan_sum[1:, 1:]))

★Step 3★

sample outer product sum
-> [[  3.  13.  35.  12.  15.]
 [ 13.  69. 230.  69.  50.]
 [ 35. 230. 925. 270.  50.]
 [ 12.  69. 270.  90.  15.]
 [ 15.  50.  50.  15. 125.]]

sample non-nan sum
-> [[3 3 2 2 2]
 [3 3 2 2 2]
 [2 2 2 1 1]
 [2 2 1 2 1]
 [2 2 1 1 2]]

sample mean vector
-> [ 4.332 17.5    6.     7.5  ]

outer product의 sum
-> [[ 23.  115.   34.5  25. ]
 [115.  462.5 270.   50. ]
 [ 34.5 270.   45.   15. ]
 [ 25.   50.   15.   62.5]]


In [13]:
# Computing covariance matrix(HW8의 3번째 방법 이용)
print("★Step 4★\n")

sample_mean_vector = sample_outer_sum[0, 1:] / np.float16(sample_nonnan_sum[0, 1:])
sample_outer_sum = sample_outer_sum[1:, 1:] / np.float16(sample_nonnan_sum[1:, 1:])

cov = sample_outer_sum - np.outer(sample_mean_vector, sample_mean_vector)
print("Covariance Matrix\n->", cov)

★Step 4★

Covariance Matrix
-> [[  4.234  39.2     8.5    -7.5  ]
 [ 39.2   156.2   165.    -81.25 ]
 [  8.5   165.      9.    -30.   ]
 [ -7.5   -81.25  -30.      6.25 ]]


### Excercise 1 - 
#### 위의 Example에서 Python으로 적용한 부분을 Spark로 적용해보자!! (40 point)

- - -

다음 데이터에 대하여 다음 과제를 수행하세요.

```
sample1 = np.array([2,np.nan,3,5])
sample2 = np.array([4,5,np.nan,10])
sample3 = np.array([7,30,9,np.nan])
```

위에서 진행된 Step 1 ~ Step 4(Python)를 Spark로 구현하세요!!

**task**

* 1 : Step 1 (10 point)

<br>

* 2 : Step 2 (10 point)

<br>

* 3 : Step 3 (10 point)

<br>

* 4 : Step 4 (10 point)

**task**

* 1 : Here is the trick: lets denote a $d$ dimensional **column vector** by $\mathbf{x} = (x_1,x_2,\ldots,x_d)$ (note that the subscript here is the index of the coordinate, not the index of the example in the training set as used above). 

The augmented vector $\mathbf{x}'$ is defined to be the $d+1$ dimensional vector $\mathbf{x}' = (1,x_1,x_2,\ldots,x_d)$. (10 point)
- - -
```
# task 1-1 output
[array([ 1.,  2., nan,  3.,  5.], dtype=float16),
 array([ 1.,  4.,  5., nan, 10.], dtype=float16),
 array([ 1.,  7., 30.,  9., nan], dtype=float16)]
 ```

In [14]:
# task 1-1 답안 작성
sample1 = np.insert(sample1, 0, 1) # sample1에 0번째 인덱스에 1을 추가
sample2 = np.insert(sample2, 0, 1) # sample2에 0번째 인덱스에 1을 추가
sample3 = np.insert(sample3, 0, 1) # sample3에 0번째 인덱스에 1을 추가
task1_1 = sc.parallelize([(sample1),(sample2),(sample3)]) # 이런 sample들을 rdd화 하여 task1_1에 저장 후 array타입으로 출력
# output
print("# task 1-1 output")
task1_1.collect()           

# task 1-1 output


[array([ 1.,  2., nan,  3.,  5.], dtype=float16),
 array([ 1.,  4.,  5., nan, 10.], dtype=float16),
 array([ 1.,  7., 30.,  9., nan], dtype=float16)]

**task**

* 2 : 

The outer product of $\mathbf{x}'$ with itself is equal to 

$$ {\mathbf{x}'}^T {\mathbf{x}'}
= \left[\begin{array}{c|ccc}
    1 &  &{\mathbf{x}'} &\\
    \hline \\
    {\mathbf{x}'}^T & &{\mathbf{x}'}^T {\mathbf{x}'} \\ \\
    \end{array}
    \right]
$$

Where the lower left matrix is the original outer product ${\mathbf{x}'}^T{\mathbf{x}'}$ and the first row and the first column are ${\mathbf{x}'}$ and ${\mathbf{x}'}^T$ respectively.

- - -

**★ sample의 outer prodcut와 outer product의 non-nan을 구할 것.!!**

```
# task 1-2
sample outer product
-> [array([[ 1.,  2., nan,  3.,  5.],
       [ 2.,  4., nan,  6., 10.],
       [nan, nan, nan, nan, nan],
       [ 3.,  6., nan,  9., 15.],
       [ 5., 10., nan, 15., 25.]], dtype=float16), array([[  1.,   4.,   5.,  nan,  10.],
       [  4.,  16.,  20.,  nan,  40.],
       [  5.,  20.,  25.,  nan,  50.],
       [ nan,  nan,  nan,  nan,  nan],
       [ 10.,  40.,  50.,  nan, 100.]], dtype=float16), array([[  1.,   7.,  30.,   9.,  nan],
       [  7.,  49., 210.,  63.,  nan],
       [ 30., 210., 900., 270.,  nan],
       [  9.,  63., 270.,  81.,  nan],
       [ nan,  nan,  nan,  nan,  nan]], dtype=float16)]

sample outer product non-nan
-> [array([[1, 1, 0, 1, 1],
       [1, 1, 0, 1, 1],
       [0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1],
       [1, 1, 0, 1, 1]]), array([[1, 1, 1, 0, 1],
       [1, 1, 1, 0, 1],
       [1, 1, 1, 0, 1],
       [0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1]]), array([[1, 1, 1, 1, 0],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0]])]
```

In [15]:
# task 1-2 답안 작성
task1_2 = task1_1.map(lambda x : (np.outer(x,x))) # 위에서 생성한 task1_1 RDD 행렬의 외적을 구한다.
outer_nonnan = task1_2.map(lambda x : (1-np.isnan(x)))
# 그런 후 np.isnan에 1을 빼서 값이 있는 인덱스는 1로 그렇지 않고 nan인 값은 0으로 표시한다.

# output
print("sample outer product\n->", task1_2.collect())
print("\nsample outer product non-nan\n->", outer_nonnan.collect())

sample outer product
-> [array([[ 1.,  2., nan,  3.,  5.],
       [ 2.,  4., nan,  6., 10.],
       [nan, nan, nan, nan, nan],
       [ 3.,  6., nan,  9., 15.],
       [ 5., 10., nan, 15., 25.]], dtype=float16), array([[  1.,   4.,   5.,  nan,  10.],
       [  4.,  16.,  20.,  nan,  40.],
       [  5.,  20.,  25.,  nan,  50.],
       [ nan,  nan,  nan,  nan,  nan],
       [ 10.,  40.,  50.,  nan, 100.]], dtype=float16), array([[  1.,   7.,  30.,   9.,  nan],
       [  7.,  49., 210.,  63.,  nan],
       [ 30., 210., 900., 270.,  nan],
       [  9.,  63., 270.,  81.,  nan],
       [ nan,  nan,  nan,  nan,  nan]], dtype=float16)]

sample outer product non-nan
-> [array([[1, 1, 0, 1, 1],
       [1, 1, 0, 1, 1],
       [0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1],
       [1, 1, 0, 1, 1]]), array([[1, 1, 1, 0, 1],
       [1, 1, 1, 0, 1],
       [1, 1, 1, 0, 1],
       [0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1]]), array([[1, 1, 1, 1, 0],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 1, 0],
       [1, 1, 

**task**

* 3 : 

Now suppose that we do the take the average of the outer product of the augmented vector and convince yourself that:
$$
\hat{E}({\mathbf{x}'}^T{\mathbf{x}'}) = \frac{1}{n} \sum_{i=1}^n {\mathbf{x}'}_i^T{\mathbf{x}'}_i
= \left[\begin{array}{c|ccc}
    1 &  &\hat{E}({\mathbf{x}'}) &\\
    \hline \\
    \hat{E}({\mathbf{x}'}^T) & &\hat{E}({\mathbf{x}'}^T{\mathbf{x}'}) \\ \\
    \end{array}
    \right]
$$

So indeed, we have produced the outer product average together with (two copies of) the average $\hat{E}({\mathbf{x}'}^T)$

- - -

**★★★★★여기서 n은 non-nan의 sum!!★★★★★**

```
# task 1-3 
outer product sum(n이 나누어지지 않은)
-> [[  3.  13.  35.  12.  15.]
 [ 13.  69. 230.  69.  50.]
 [ 35. 230. 925. 270.  50.]
 [ 12.  69. 270.  90.  15.]
 [ 15.  50.  50.  15. 125.]]

outer product non-nan sum
-> [[3 3 2 2 2]
 [3 3 2 2 2]
 [2 2 2 1 1]
 [2 2 1 2 1]
 [2 2 1 1 2]]

outer product sum(n이 나누어진)
-> [ 23.  115.   34.5  25. ]

sample mean vector
-> [ 4.332 17.5    6.     7.5  ]
```

In [16]:
# task 1-3 답안 작성
task1_3_outer_sum = task1_2.map(lambda x : np.nan_to_num(x)).reduce(lambda x,y : x+y)
#np.nan_to_num() 함수를 사용해 task1_2에서 구한 행렬들 중 nan인 값을 map을 사용하여 0으로 바꾼 후 reduce를 통하여 외적한 행렬을 합한다.

task1_3_nonnan_sum = outer_nonnan.reduce(lambda x,y : x+y)
# 위에서 구한 outer_nonnan의 행렬을 reduce를 사용해 합한다.

task_outer_sum = task1_2.map(lambda x : np.nan_to_num(x)[1:,1:]).reduce(lambda x,y: x+y)
task_nonnan_sum = task1_2.map(lambda x : (1-np.isnan(x)[1:,1:])).reduce(lambda x,y : x+y)
sample_outer_product = (task_outer_sum / np.float16(task_nonnan_sum))[0]
# task1_3_outer_sum의 2행 2열 부터의 값을 task1_3_nonnan_sum의 2행2열 부터의 값까지 나눈다 1행 1열은 mean을 구하기 위해 임의로 1을 넣어준
# 것이기 때문에 sample_outer_product를 구할 때에는 불필요 하므로 제거한다.

task_outer_sum_mean = task1_2.map(lambda x : np.nan_to_num(x)[0,1:]).reduce(lambda x,y: x+y)
task_nonnan_sum_mean = task1_2.map(lambda x : (1-np.isnan(x)[0,1:])).reduce(lambda x,y : x+y)
sample_mean_vector = task_outer_sum_mean / np.float16(task_nonnan_sum_mean)
# 평균을 구하기 위해 task1_3_outer_sum의 1행 2열의 값 부터 열의 끝까지의 합을 task1_3_nonnan_sum의 1행 2열부터 끝까지의 열의 값으로 나눠준다.
# output
print("outer product sum(n이 나누어지지 않은)\n->", task1_3_outer_sum)
print("\nouter product non-nan sum\n->", task1_3_nonnan_sum)
print("\nouter product sum(n이 나누어진)\n->", sample_outer_product)
print("\nsample mean vector\n->", sample_mean_vector)

outer product sum(n이 나누어지지 않은)
-> [[  3.  13.  35.  12.  15.]
 [ 13.  69. 230.  69.  50.]
 [ 35. 230. 925. 270.  50.]
 [ 12.  69. 270.  90.  15.]
 [ 15.  50.  50.  15. 125.]]

outer product non-nan sum
-> [[3 3 2 2 2]
 [3 3 2 2 2]
 [2 2 2 1 1]
 [2 2 1 2 1]
 [2 2 1 1 2]]

outer product sum(n이 나누어진)
-> [ 23.  115.   34.5  25. ]

sample mean vector
-> [ 4.332 17.5    6.     7.5  ]


**task**

* 4 :

Computing covariance matrix

\begin{align*}
\text{Cov}(\mathbf{x}) &= \hat E({\mathbf{x}'}^T{\mathbf{x}'}) \ - \hat{E}({\mathbf{x}'})^T\hat{E}({\mathbf{x}'})
\end{align*}

- - -


```
# task 1-4
covariance matrix
-> [[  4.234  39.2     8.5    -7.5  ]
 [ 39.2   156.2   165.    -81.25 ]
 [  8.5   165.      9.    -30.   ]
 [ -7.5   -81.25  -30.      6.25 ]]
```

In [17]:
# task 1-4 답안 작성
cov = sample_outer_product - np.outer(sample_mean_vector,sample_mean_vector)
# nonnan으로 나누어진 sample행렬에서 sample행렬의 평균 백터의 외적을 빼서 covariance marix를 구한다.

# output
print("covariance matrix\n->", cov)

covariance matrix
-> [[   4.234   39.2      8.5     -7.5  ]
 [ -52.8   -191.2    -70.5   -106.25 ]
 [  -3.      10.      -1.5    -20.   ]
 [  -9.5    -16.25   -10.5    -31.25 ]]


### Excercise 2 - 
####  (60 point)

- - -

다음 데이터에 대하여 다음 과제를 수행하세요.

- regular_float32.zip : KBO에서 활약한 타자들의 역대 정규시즌 성적을 포함하여 타자의 고유 아이디, 타자의 이름, 팀 이름, 기록, 년도 등을 포함한 parquet 파일 입니다.

```
# regular_float32 예시
<regular_float32.show(5)>
+---------+------+----+--------------------+----+
|batter_id|  name|team|              values|year|
+---------+------+----+--------------------+----+
|      297|진갑용|삼성|[17 D9 8E 3E 00 0...|2008|
|      297|진갑용|삼성|[68 91 6D 3E 00 0...|2009|
|      297|진갑용|삼성|[F0 A7 86 3E 00 0...|2010|
|      297|진갑용|삼성|[A8 C6 8B 3E 00 0...|2011|
|      297|진갑용|삼성|[1B 2F 9D 3E 00 0...|2012|
+---------+------+----+--------------------+----+
only showing top 5 rows

<schema>
root
 |-- batter_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- team: string (nullable = true)
 |-- values: binary (nullable = true)
 |-- year: long (nullable = true)

<row의 수(sample의 수)> :2454
```

- - -

##### regular_float32의 values에 대한 정보
    A. values는 byte array로 각각의 값은 float32로 이루어져 있습니다.
    B. values에 포함된 값은 아래와 같습니다(index 순서와 동일).
       [0]타율 [1]출전게임수 [2]타수(타석-볼넷-사구-희생번트-희생플라이)
       [3]득점 [4]안타(1루타+2루타+3루타+홈런) [5]2루타 [6]3루타
       [7]홈런 [8]루타 수 [9]타점 [10]도루 성공 [11]도루 실패 [12]볼넷
       [13]사구(몸에 맞는 볼) [14]삼진아웃 [15]병살타 [16]장타율 [17]출루율
       [18]에러
    C. values의 포함된 값은 nan이 존재할 수 있습니다.
    D. values를 ndarray로 변환시에는 dtype="float32"를 사용하세요.
    
    
- - -
##### Excercise 2에서 포함되어야 할 사항
    A. 모든 작업은 RDD로 수행합니다.
    B. Excercise 1에서 진행했던 모든 단계를 함수화 합니다.
        (1) 함수의 이름, 기능은 자유롭게 설정합니다(1개 또는 여러개를 사용해도 됩니다)
        (2) 단, 작성된 함수를 종합하여 실행하는 함수를 만들어야 합니다(ex. calcCov)
    C. check_symmetric :  True, Exercise2 shape :  (19, 19) 가 결과로 출력되는지 확인하세요.
    D. covariance matrix를 구한 후, regular_cov.txt 파일로 저장하여 과제란에 같이 제출합니다.

- - -
```
# Exercise 2 output
check_symmetric :  True

Exercise2 shape :  (19, 19)

Exercise2 result
-> [[9.68648770e-03 2.31904584e+00 8.71907655e+00 1.45815861e+00
  2.81985552e+00 4.96244952e-01 4.60792588e-02 3.05693491e-01
  4.32533845e+00 1.44053534e+00 2.43099885e-01 9.82517298e-02
  9.88237934e-01 1.42195695e-01 1.39666974e+00 1.98392683e-01
  1.39535328e-02 9.58807760e-03 1.26492938e-01],
                ...(중간 생략)...
 [1.26492938e-01 1.19444102e+02 4.45646963e+02 6.70115876e+01
  1.28225598e+02 2.35202158e+01 2.19940931e+00 1.39919460e+01
  1.98120470e+02 6.50261108e+01 1.26920823e+01 5.25860146e+00
  4.43497757e+01 8.11945502e+00 7.72257562e+01 1.05942064e+01
  2.23718629e-01 1.35275737e-01 2.10159326e+01]]
 ```
(regular_cov.txt 예시)

<p><img alt="" src="https://drive.google.com/uc?id=1bIZAMXlyxw_WtZtiYaDBmlCH6MkZUDil" style="height:324px; width:900px" /></p>

In [18]:
import urllib.request
from os.path import exists

!mkdir regular.parquet


f = urllib.request\
.urlretrieve ("https://docs.google.com/uc?export=download&id=1GUP0n9aybONtLkGre8GnBMBpMgR777Wy",
              "./regular_float32.zip")


!unzip regular_float32.zip -d regular.parquet/
!rm -rf regular_float32.zip


"""
f = urllib.request\
.urlretrieve ("https://docs.google.com/uc?export=download&id=1fbTRIL2HGRYss0FnrwcM1lgZ0nm0pjVE",
              "./regular_float16.zip")
"""

mkdir: cannot create directory ‘regular.parquet’: File exists
Archive:  regular_float32.zip
replace regular.parquet/._SUCCESS.crc? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


'\nf = urllib.request.urlretrieve ("https://docs.google.com/uc?export=download&id=1fbTRIL2HGRYss0FnrwcM1lgZ0nm0pjVE",\n              "./regular_float16.zip")\n'

In [19]:
# data 읽기
data = sqlContext.read.parquet("./regular.parquet/")

data.show(5)
data.printSchema()
data.count()

+---------+------+----+--------------------+----+
|batter_id|  name|team|              values|year|
+---------+------+----+--------------------+----+
|      297|진갑용|삼성|[17 D9 8E 3E 00 0...|2008|
|      297|진갑용|삼성|[68 91 6D 3E 00 0...|2009|
|      297|진갑용|삼성|[F0 A7 86 3E 00 0...|2010|
|      297|진갑용|삼성|[A8 C6 8B 3E 00 0...|2011|
|      297|진갑용|삼성|[1B 2F 9D 3E 00 0...|2012|
+---------+------+----+--------------------+----+
only showing top 5 rows

root
 |-- batter_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- team: string (nullable = true)
 |-- values: binary (nullable = true)
 |-- year: long (nullable = true)



2454

In [80]:
# Exercise 2 답안 작성  

def calc_func(sum_list):
    a = data.rdd.map(lambda row : np.insert(row, 0, 1))
    # 주어진 data를 rdd로 변형 시켜 row로 받아 0번째 인덱스에 1을 추가
    b = a.map(lambda x : (np.array([x[4][0],x[4][1],x[4][2],x[4][3],x[4][4],x[4][5],x[4][6],x[4][7],x[4][8],x[4][9],x[4][10],x[4][11],x[4][12],x[4][13],x[4][14],x[4][15],x[4][16],x[4][17],x[4][18]],dtype='float32')))
    # 생성한 rdd를 인덱싱하여 value가 있는 4번째 인덱스에 접근하여 그 안에 있는 인덱스의 각각의 요소를 ndarray로 dtype을 'float32'로 변환한다.
    outer_sum = b.map(lambda x : np.outer(x,x)).map(lambda x : np.nan_to_num(x)).reduce(lambda x,y : x+y)
    # 그 변환된 value들의 외적을 구한 후 nan인 값들을 0으로 변환해 주는 np.nan_to_num() 함수를 사용해 주고 reduce를 사용하여 그 행렬들을 합친다.
    nonnan_outer_sum = b.map(lambda x : (1-np.isnan(x))).reduce(lambda x,y : x+y)
    # nan이 아닌 값을 합한다.
    outer_vector = (outer_sum[1:,1:] / np.float16(nonnan_outer_sum[1:]))
    #outer_sum의 2행2열의 값부터 행렬의 끝까지를 nonnan_outer_sum의 2행의 값으로 나눠 nonnan으로 나눠진 outer_vector를 구한다.
    mean_vector = outer_sum[0:] / np.float16(nonnan_outer_sum[0:])
    #nonnan으로 나누어지지 않은 outer_sum의 행렬 1행부터 끝 행렬까지를 nonnan_outer_sum의 행렬으로 나눠 평균 벡터 mean_vector를 구한다. 
    return {"Mean_vector" : mean_vector, "Outer_vector" : outer_vector}
    
def calcCov(DataFrame):
    # 위 함수를 활용하여 DataFrame을 calc_func 입력 변환 사전작업
    Result_cacl = calc_func(data) # data를 매개변수로 받은 calc_func함수를 호출하여 Result_cacl에 저장한다
    Cov = outer_sum - np.outer(mean_vector,mean_vector)
    # n으로 나누어 지지 않은행렬 outer_sum을 평균 벡터(mean_vector)의 외적으로 뺀다.
    
    return Cov

In [81]:
# Exercise 2 output
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

Exercise2_Cov = calcCov(data)
print("check_symmetric : ", check_symmetric(Exercise2_Cov))
print("\nExercise2 shape : ", Exercise2_Cov.shape)
print("\nExercise2 result\n->", Exercise2_Cov)

# textFile로 저장 후 주피터 노트북과 같이 제출
np.savetxt("regular_cov.txt", Exercise2_Cov, delimiter="\t")

check_symmetric :  True

Exercise2 shape :  (19, 19)

Exercise2 result
-> [[-3.5758848e+08 -2.8115619e+08 -2.6172613e+08 -1.2684158e+08
   0.0000000e+00  0.0000000e+00 -3.0244221e+08 -1.3487427e+08
   0.0000000e+00 -5.7528216e+07 -2.8747539e+08 -1.3607114e+08
   0.0000000e+00  0.0000000e+00 -2.2492155e+08 -1.2879888e+08
   0.0000000e+00  0.0000000e+00 -2.4099474e+08]
 [-2.8115619e+08 -2.0409245e+08 -1.9846790e+08 -9.6149936e+07
   0.0000000e+00  0.0000000e+00 -2.2824370e+08 -1.0223378e+08
   0.0000000e+00 -4.3653016e+07 -2.1783456e+08 -1.0313334e+08
   0.0000000e+00  0.0000000e+00 -1.7055355e+08 -9.7519840e+07
   0.0000000e+00  0.0000000e+00 -1.8239234e+08]
 [-2.6172613e+08 -1.9846790e+08 -1.7770552e+08 -8.7655056e+07
   0.0000000e+00  0.0000000e+00 -2.0927264e+08 -9.3550576e+07
   0.0000000e+00 -3.9107312e+07 -1.9960214e+08 -9.4687528e+07
   0.0000000e+00  0.0000000e+00 -1.5633586e+08 -8.9508096e+07
   0.0000000e+00  0.0000000e+00 -1.6775102e+08]
 [-1.2684158e+08 -9.6149936e+07 -8.765

### ㅎㅎ~
![hi](https://contents.vryjam.com/g1TwHtUsHsPFaeQ.gif)